# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import json
from pprint import pprint


# Import API key
from api_keys import g_key
import gmaps


ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [52]:
weather_df = pd.read_csv('../output_data/cities.csv', index_col=0)
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City_ID,,,,,,,,,
0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...
543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [53]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#Define variables for Heat map
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure(center=(20.0,0.0),zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False, max_intensity=50, point_radius = 2)
fig.add_layer(heat_layer)

fig

NameError: name 'gmaps' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [56]:
#Drop NaN results
weather_df = weather_df.dropna()

#Define weather criteria
weather_criteria = weather_df[(weather_df['Max Temp'] > 55) & 
                              (weather_df['Max Temp']< 70) & 
                              (weather_df['Humidity'] < 40) &
                              (weather_df['Wind Speed'] < 10)&
                              (weather_df['Cloudiness'] < 50)]

weather_criteria.reset_index()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,107,whitehorse,40,CA,1558378774,36,60.72,-135.06,64.00,3.36
1,156,narsaq,20,GL,1558378784,28,60.91,-46.05,57.20,9.17
2,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50
3,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20
4,261,fort frances,1,CA,1558378834,38,48.61,-93.40,63.00,9.17
5,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50
6,340,la libertad,0,PE,1558378855,36,-16.38,-71.56,64.40,8.05
7,362,datong,0,CN,1558378858,23,40.09,113.30,55.32,9.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [58]:
#Create new data frame with specific columns from criteria results
hotel_df = pd.DataFrame(weather_criteria,columns=['City','Country','Lat','L'])

#Add a new column for "Hotel Name"
hotel_df['Hotel Name']= ""

#Drop NaN results
hotel_df.dropna()

hotel_df.reset_index()

,City_ID,City,Country,Lat,Lng,Hotel Name
0,107,whitehorse,CA,60.72,-135.06,
1,156,narsaq,GL,60.91,-46.05,
2,167,yumen,CN,40.29,97.04,
3,259,hami,CN,42.84,93.51,
4,261,fort frances,CA,48.61,-93.40,
5,299,jiuquan,CN,39.73,98.49,
6,340,la libertad,PE,-16.38,-71.56,
7,362,datong,CN,40.09,113.30,


In [59]:
hotel_df = df_trimed.copy()
hotel_df['Hotel Name'] = ''
hotel_names = []

In [62]:
#Create empty list for Hotel Name column and parameters to search
hotel_name=[]
target_type = "hotel"
radius = 5000

#Define parameters
params = {
    "types": target_type,
    "radius": radius,
    "key": g_key
}

#For loop to search for hotels in locations meeting weather criteria
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
      
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_json=response.json()

    try: 
        hotel_name.append(response_json['results'][0]['name'])
    
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    
        print(f"The nearest hotel to {row[0]} is {response_json['results'][0]['name']}")  
    
    except (KeyError, IndexError):
        print("Cannot locate a hotel - skipping.")

Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.
Cannot locate a hotel - skipping.


In [63]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [64]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(20.0,0.0),zoom_level=1)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display Map
fig

NameError: name 'gmaps' is not defined